<a href="https://colab.research.google.com/github/pontikos/retinal_imaging/blob/master/create_filters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [0]:
imgpath = './combine/female_correct.png'

In [0]:
scale = (450,450)
cropcenter = (scale[0]/2, scale[1]/2)
bopcenter = cropcenter
clicking = False
zoomed = False
library = []

In [0]:
def click_and_zoom(event, x, y, flags, param):
    global cropcenter, clicking, zoomed
    
    if event == cv2.EVENT_LBUTTONDOWN:
        if not zoomed:
            cropcenter = (x,y)
        clicking = True

        
    elif event == cv2.EVENT_LBUTTONUP:
        clicking = False

In [0]:
img = cv2.imread(imgpath,1)

In [0]:
cv2.namedWindow('Microaneurysm_Labelling')
cv2.setMouseCallback('Microaneurysm_Labelling', click_and_zoom)

height, width, channels = img.shape
missalignment = width-height
offset = int(missalignment/2)

radiusXFilter = 5
radiusYFilter = 5

radiusXCrop = 20
radiusYCrop = 20

radiusXZoom = ((radiusXFilter/radiusXCrop) * scale[0]) /2
radiusYZoom = ((radiusYFilter/radiusYCrop) * scale[0]) /2

if(missalignment < 0):
    print("ERROR: negative missalignment")
if(missalignment % 2 == 1):
    print("ERROR: Uneven missalignment")
    


img = img[0:height, offset:(width-offset)]
img = cv2.resize(img, scale)
bimg = img.copy()
zimg = img.copy()

while True:
    
    if not zoomed:
        img = bimg.copy()
    else:
        img = zimg.copy()
    
    centerX,centerY = cropcenter
    
    #TODO: Check Borders
    
    if not zoomed:
        minXFilter,maxXFilter = int(centerX-radiusXFilter), int(centerX+radiusXFilter)
        minYFilter,maxYFilter = int(centerY-radiusYFilter), int(centerY+radiusYFilter)
    else:
        minXFilter,maxXFilter = int(centerX-radiusXZoom), int(centerX+radiusXZoom)
        minYFilter,maxYFilter = int(centerY-radiusYZoom), int(centerY+radiusYZoom)
        
        
    
    imgRec = cv2.rectangle(img, (minXFilter,minYFilter), (maxXFilter,maxYFilter), (0,255,0), 2)
    
    cv2.imshow('Microaneurysm_Labelling', imgRec)
    
    key = cv2.waitKeyEx(1) #& 0xFF
    
    #TODO: Zoom functionality
    
    if key == ord('i'):
        print('i')
        zoomed = True

        #TODO: Check Borders
        minXCrop, maxXCrop = centerX - radiusXCrop, centerX + radiusXCrop
        minYCrop, maxYCrop = centerY - radiusYCrop, centerY + radiusYCrop
        img = bimg.copy()
        img = img[minYCrop:maxYCrop, minXCrop:maxXCrop]
        img = cv2.resize(img, scale)
        
        zimg = img.copy()
        
        bopcenter = cropcenter
        cropcenter = (scale[0]/2, scale[1]/2)
        
        
    if key == ord('o'):
        print('o')
        zoomed = False
        
        img = bimg.copy()
        cropcenter = bopcenter
        
    if key == ord('r'):
        print('r')
        minXFilter,maxXFilter = int(centerX-radiusXFilter), int(centerX+radiusXFilter)
        minYFilter,maxYFilter = int(centerY-radiusYFilter), int(centerY+radiusYFilter)
        
        reference = bimg[minYFilter:maxYFilter, minXFilter:maxXFilter]
        
        library.append(reference)
        
    if key == 2424832: #Left on Windows
        print('Left')
        cropcenter = cropcenter[0]-1, cropcenter[1]
        
    if key == 2555904: #Right on Windows
        print('Right')
        cropcenter = cropcenter[0]+1, cropcenter[1]
        
    if key == 2490368: #Up on Windows
        print('Up')
        cropcenter = cropcenter[0], cropcenter[1]-1
        
    if key == 2621440: #Down on Windows
        print('Down')
        cropcenter = cropcenter[0], cropcenter[1]+1
        
    if key == 27: #Esc
        break
    
    #Use this code to check for correct keycodes on your system    
    #if(key == -1):
    #    continue
    #else:
    #    print(key)
        
with open('referenceLib.pkl', 'wb') as f:
    pickle.dump(library, f)
        
cv2.destroyAllWindows()